In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pickle
import h5py
import seaborn as sns
import pandas as pd
from tensorboard.backend.event_processing import event_accumulator
from scipy.stats import pearsonr
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.CleanSoundsDataset import CleanSoundsDataset
from data.NoisyDataset import NoisyDataset, FullNoisyDataset

# PNet parameters

In [2]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP
pnet_name = 'pnet'
chckpt = 50

In [3]:
n_timesteps = 5
layers = ['conv1', 'conv2', 'conv3', 'conv4_W', 'conv5_W']

# Paths to relevant directories

In [4]:
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
activations_dir = f'{engram_dir}activations_pnet_all/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
pickle_dir = f'{engram_dir}pickles/'

In [5]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')

Device: cpu


# Helper functions to save activations

In [6]:
bgs = ['pinkNoise', 'AudScene', 'Babble8Spkr']
snrs = [-9.0, -6.0, -3.0, 0.0, 3.0]

In [7]:
dead_units = {}
for l in layers:
    dead_units[l] = []

In [8]:
from functools import reduce

In [9]:
dead_units = {}
for bg in bgs:
    for snr in snrs:
        for l in layers:
            dead_units[l] = []
        results_path = f'{activations_dir}{bg}_snr{int(snr)}.hdf5'
        results = h5py.File(results_path, 'r')
        labels = np.array(results['label'])
        for i in range(labels.size):
            for l in layers:
                activs = []
                for t in range(1): #n_timesteps):
                    noisy_activ = results[f'{l}_{t}_activations'][i]
                    activs.append(noisy_activ.flatten())
                activs = np.array(activs)
                dead_u = np.argwhere(np.sum(activs, axis=0)==0)
                dead_units[l].append(dead_u)
        for l in layers:
            x = reduce(np.intersect1d, dead_units[l])
            dead_units[l] = [x.tolist()]

In [12]:
for l in layers:
    dead_units[l] = dead_units[l][0]

In [ ]:
from functools import reduce
undead_units = {}
for l in dead_units.keys():
    n_units = reduce(np.multiply, n_units_per_layer[l])
    undead = np.ones(n_units).astype(bool)
    undead[dead_units[l]] = False
    undead_units[l] = undead
undead_units['fc6_W'] = np.ones(4096).astype(bool)

In [15]:
with open(f'{pickle_dir}dead_units.p', 'wb') as f:
    results = {}
    results['dead_units'] = dead_units
    results['undead_units'] = undead_units
    pickle.dump(results, f)